In [11]:
import requests
import time

DEFAULT_HEADERS = {
    'user-agent': 'wasp',
    'referer': 'https://space.bilibili.com/'
}
        

class Net(object):
    """封装了 GET 和 POST 两个 HTTP 请求的相关方法，具体用法和 requests 的一样，不过把 requests.get 换成了 Net.get"""
    @staticmethod
    def request(method: str, url: str, headers: dict=None, params: dict=None, data: dict=None, **kwargs) -> requests.models.Response:
        _headers = {'user-agent': 'wasp', 'referer': url}
        
        if headers is None:
            headers = _headers
        
        if 'user-agent' not in headers:
            headers.update({'user-agent': 'wasp'})
            
        return requests.request(method, url, headers=headers, params=params, data=data, **kwargs)
    
    @staticmethod
    def get(url: str, headers: dict=None, params: dict=None, **kwargs) -> requests.models.Response:
        return Net.request('GET', url, headers=headers, params=params, data=None, **kwargs)
    
    @staticmethod
    def post(url: str, headers: dict=None, data: dict=None, **kwargs) -> requests.models.Response:
        return Net.request('POST', url, headers=headers, params=None, data=data, **kwargs)

    
class BiliFansTool(object):
    """目前仅实现了批量移除粉丝功能"""
    def __init__(self, SESSDATA, BILI_JCT):
        self.headers = dict(DEFAULT_HEADERS)
        self.headers.update({'cookie': f'SESSDATA={SESSDATA}; BILI_JCT={BILI_JCT};'})
        
        self.BILI_JCT = BILI_JCT
        self.SESSDATA = SESSDATA
        
    def remove_one(self, fid: str) -> None:
        """移除一个指定的粉丝
        :param: 粉丝的 UID
        """
        api = 'https://api.bilibili.com/x/relation/modify'
        
        data = {
            'fid': fid,
            'act': 7,
            're_src': 11,
            'jsonp': 'jsonp',
            'csrf': self.BILI_JCT
        }
        
        print(f'Removing UID: {fid}')
        
        ret = Net.post(api, data=data, headers=self.headers)
        
        print(ret.json())
        
    def remove(self, fid_list: list) -> None:
        """移除多个粉丝
        :param: 一个 UID 列表
        """
        if not isinstance(fid_list, list):
            fid_list = [fid_list]
        
        for fid in fid_list:
            self.remove_one(fid)
            print('休眠 2s.')
            time.sleep(2) # 设置你的休眠时间
    
    def remove_one_page(self, mid: str, pn: int) -> bool:
        """移除一页粉丝（默认是20个粉）
        :param: 你的 UID
        :pn:    要移除的页数
        
        :return: 一个 bool 值，表示粉丝数是否为 0 
        """
        followers = self.get_followers(mid, pn)
        
        for item in followers:
            mid, uname = item['mid'], item['uname']
            print()
            print(f'再见，{uname}')
            self.remove(mid)
            print()
        
        print()
        return len(followers) == 0 # 是否是空页
            
    def remove_many_pages(self, mid: str, begin: int, end: int) -> None:
        """移除多页粉丝
        :param: mid 你的 UID
        :param: being 起始页
        :param: end 结束页
        """
        for pn in range(begin, end):
            print(f'开始移除第 {pn} 页粉丝')
            
            # 类似 list 的 pop(0)，把第一个元素 pop 了之后第二个元素就变成第一个元素了
            # 这里移除了第一页的粉丝之后，第二页的粉丝就变成第一页的了，所以这里 pn 要写成常量
            if self.remove_one_page(mid, 1): 
                print(f'第 {pn} 页没粉丝了')
                break
            
            print('休眠 3s')
            time.sleep(3)
            
    def get_followers(self, mid: str, pn: int, ps: int=20, order: str='desc') -> list:
        """获取用户的粉丝列表
        :param: mid 要获取的用户的 UID
        :param: pn 获取的页数
        :param: ps 每页返回的粉丝数
        :param: order 排序方式 
        """
        api = 'https://api.bilibili.com/x/relation/followers'

        params = {
            'vmid': mid,
            'pn': pn,
            'ps': ps,
            'order': order,
            'order_type': 'attention',
            'jsonp': 'jsonp'
        }

        ret = Net.get(api, params=params, headers=self.headers).json()

        if ret['code'] == 0:
            return ret['data']['list'] # 是个 list，其中每一项的 mid 就是我们需要的值

        print(f'获取粉丝列表失败: {ret}')

        return []

_ = """
考虑到反爬，对每次请求的间隔进行了设置。移除粉丝是每个停止 2s，请求粉丝数据是每个停止 3s
于是可以得出清粉需要的总耗时 T:

T = 你的粉丝数 * 2 + 你的粉丝数 / 20 s

后面那个式子算的是最多共有几页粉丝，这里的页数是 20粉 / 页


另外既然是一键清粉，所以其实移除单个粉丝之类的方法其实并不常用，一般用 remove_many_pages 移除全部粉丝，其中 end 参数可以很大，程序会在没有粉丝
之后自动停止

一个奇妙的点：
清粉 API 返回的数据无法判断用户是否是登录的用户，就是说假设你的 UID=3，你可以清 UID=4 的人的粉，尽管这样根本没有实际作用
"""

In [12]:
# 填入你的 COOKIE 中对应的值
SESSDATA = ''
BILI_JCT = ''

tool = BiliFansTool(SESSDATA=SESSDATA, BILI_JCT=BILI_JCT)

tool.remove_many_pages('999', begin=1, end=100)

In [ ]:
# 2021年3月4日23:32:38 睡觉